In [1]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [2]:
train_data=data[:1000]
train_target=target[:1000]

test_data=data[1000:]
test_target=target[1000:]

print(train_data.shape)
print(test_data.shape)
print(train_target.shape)
print(test_target.shape)

(1000, 50, 1)
(209, 50, 1)
(1000, 1)
(209, 1)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,MaxPooling1D,Dense,Dropout,Activation,Flatten
import numpy as np

from tensorflow.keras.optimizers import Adam,Adagrad,Adadelta

def build_model(parameters):
    
    model = Sequential()
    
    for i in range(parameters.Int('num_layers', 2, 7)):

        if(i==0):
            model.add(Conv1D(filters=parameters.Int('#CNN layer' + str(i),min_value=240,max_value=300,step=10),
                            kernel_size=3,activation=parameters.Choice('activation_function '+str(i),['relu','sigmoid','tanh']),input_shape=(50,1)))
            
            model.add(Dropout(parameters.Choice('drop_prob 1',[0.2,0.3,0.4,0.5])))
            
            model.add(MaxPooling1D(pool_size=2))
        
        else:
            model.add(Conv1D(filters=parameters.Int('#CNN layer' + str(i),min_value=72,max_value=120,step=8),
                                   kernel_size=3,activation=parameters.Choice('activation_function '+str(i),['relu','sigmoid','tanh'])))
            
            model.add(Dropout(parameters.Choice('drop_prob 2',[0.2,0.3,0.4,0.5])))
            model.add(MaxPooling1D(pool_size=2))
            
    model.add(Flatten())
    model.add(Dense(1, activation='linear'))
    
    model.compile(optimizer=parameters.Choice('optmz', ['adam', 'adaDelta', 'adaGrad']),loss=parameters.Choice('loss f', ['mse', 'mae']))
    #model.compile(optimizer=Adam(parameters.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),loss='mse')
    
    return model

In [6]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(build_model,objective='val_loss',max_trials=5,executions_per_trial=3,directory='project2',
                     project_name='CNN - ModelTune')

c:\Users\USER\Music\DeepL\myenvdeep\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 7, 'step': 1, 'sampling': 'linear'}
#CNN layer0 (Int)
{'default': None, 'conditions': [], 'min_value': 240, 'max_value': 300, 'step': 10, 'sampling': 'linear'}
activation_function 0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
drop_prob 1 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4, 0.5], 'ordered': True}
#CNN layer1 (Int)
{'default': None, 'conditions': [], 'min_value': 72, 'max_value': 120, 'step': 8, 'sampling': 'linear'}
activation_function 1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
drop_prob 2 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4, 0.5], 'ordered': True}
optmz (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'adaDelta', 'adaGrad'], 'ordered': False}
loss f (Ch

In [8]:
tuner.search(train_data,train_target,epochs=10,validation_data=(test_data, test_target))

Trial 5 Complete [00h 00m 09s]
val_loss: 0.42667309443155926

Best val_loss So Far: 0.42667309443155926
Total elapsed time: 00h 00m 30s


In [9]:
tuner.results_summary()

Results summary
Results in project2\CNN - ModelTune
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 4 summary
Hyperparameters:
num_layers: 2
#CNN layer0: 260
activation_function 0: relu
drop_prob 1: 0.5
#CNN layer1: 80
activation_function 1: sigmoid
drop_prob 2: 0.5
optmz: adam
loss f: mae
#CNN layer2: 80
activation_function 2: tanh
#CNN layer3: 120
activation_function 3: tanh
#CNN layer4: 88
activation_function 4: tanh
Score: 0.42667309443155926

Trial 0 summary
Hyperparameters:
num_layers: 2
#CNN layer0: 290
activation_function 0: relu
drop_prob 1: 0.2
#CNN layer1: 120
activation_function 1: sigmoid
drop_prob 2: 0.3
optmz: adaDelta
loss f: mae
Score: 0.7110349436601003

Trial 2 summary
Hyperparameters:
num_layers: 2
#CNN layer0: 270
activation_function 0: relu
drop_prob 1: 0.3
#CNN layer1: 80
activation_function 1: relu
drop_prob 2: 0.5
optmz: adaDelta
loss f: mae
#CNN layer2: 88
activation_function 2: relu
#CNN layer3: 88
activation_function 3: relu
#CNN la

In [10]:
best_models = tuner.get_best_models()
print(best_models[0].summary()) # parameter tuning final update (1d CNN)

c:\Users\USER\Music\DeepL\myenvdeep\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 48, 260)        │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 48, 260)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 24, 260)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 22, 80)         │        62,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 22, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 11, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 880)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           881 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,401 (251.57 KB)

 Trainable params: 64,401 (251.57 KB)

 Non-trainable params: 0 (0.00 B)

None
